# 日本語データセットをチャンク分割し、Embeddings に変換して Azure Cognitive Search のベクトルインデックスに登録するサンプル

Azure Cognitive Search のベクトル検索を行うために、以下を行うサンプルです。
1. 戦国武将の Wikipedia ページをチャンクに分割
1. チャンクごとに Embeddings を生成
1. Azure Cognitive Search のベクトルインデックスに登録
 

# 1. 日本語データセットをチャンクに分割

In [ ]:
!pip install openai tiktoken langchain

In [ ]:
import warnings
import tiktoken
import pandas as pd
import glob
import os
import base64
import re

from langchain.text_splitter import RecursiveCharacterTextSplitter
warnings.filterwarnings('ignore')

In [ ]:
#!pip install pyarrow
#import pyarrow.parquet as pq

## 単一ファイルの動作確認

In [ ]:
f = open('./data/源頼朝.txt', 'r', encoding='UTF-8')

data = f.read()
print(data)

f.close()

## ドキュメントのトークン数を計測
OpenAI モデルのトークン数を正確に数えるためには、[tiktoken](https://github.com/openai/tiktoken) ライブラリを利用します。モデルごとに利用するトークナイザーが異なるため詳しくは[こちら](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb)を参照してください。

In [ ]:
enc = tiktoken.get_encoding("cl100k_base")
print(len(enc.encode(data)))

## テキストをチャンクに分割
通常 1 つのドキュメントに含まれるトークン数は 1 度のコンテキストに指定できる最大トークン数をゆうに超えます。今回はドキュメントの文を指定したトークン数以下のチャンクに分割します。

- テキストの分割方法<br>
RecursiveCharacterTextSplitter はチャンクが十分に小さくなるまで、順番に分割します。
- チャンクサイズの測定方法<br>
tiktoken ライブラリのトークナイザーを使用してトークン数を正確に測定します。

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=1000, 
    chunk_overlap=50
)

In [ ]:
chunk = text_splitter.split_text(data)
chunk

In [ ]:
#チャンク数
len(chunk)

In [ ]:
#チャンク[0]の中身
chunk[0]

In [ ]:
#トークン数
print(len(enc.encode(chunk[0])))

In [ ]:
#チャンクごとのトークン数確認
tokencounter = 0
for i, text in enumerate(chunk):
    print(i,len(enc.encode(text)))
    tokencounter = tokencounter + len(enc.encode(text))

print(tokencounter)

## ファイル一括処理
ドキュメントをチャンクに分割する方法を確認したら、実際に複数のデータセットを一括処理でチャンクごとのテキストファイルに分割します。

In [ ]:
output_dir_path = './data/output'
os.mkdir(output_dir_path)

In [ ]:
enc = tiktoken.get_encoding("cl100k_base")
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=1000, 
    chunk_overlap=50
)

#テキストファイルを読み込んで、指定のトークン数のチャンクファイルに分割します。
def splitChunkFile(filepath):
    f = open(filepath, 'r', encoding='UTF-8')
    data = f.read()
    chunk = text_splitter.split_text(data)

    #chunk単位でループ
    for i, chunkedtext in enumerate(chunk):
        
        dirname = os.path.dirname(filepath)
        basename = os.path.splitext(os.path.basename(filepath))[0]
        outputfilepath = dirname + "/output/" + basename + "-" + str(i) + ".txt"
        
        print(i, len(enc.encode(chunkedtext)), outputfilepath)
        with open(outputfilepath, 'w', encoding='UTF-8') as fo:
            fo.write(chunkedtext)

        fo.close()
    f.close()
   
    return

In [ ]:
for p in glob.glob('./data/*.txt'):
    splitChunkFile(p)

生成されたテキストファイルを Azure Blob Storage にアップロードして Azure Cognitive Search に登録することもできます。

# 2. 必要なデータフレームを作成

In [ ]:
df = pd.DataFrame([], columns=['id', 'content', 'sourcepage'])

In [ ]:
df

In [ ]:
def makeDataFrame(filepath):
    f = open(filepath, 'r', encoding='UTF-8')
    data = f.read()
    content = " ".join(data.splitlines())
    
    filename = os.path.basename(filepath)
    enc_id = base64.urlsafe_b64encode(filename.encode())
    
    return {'id': enc_id.decode(), 'content': content, 'sourcepage': filename}

チャンク分割済みテキストファイルを DataFrame の行としてロード

In [ ]:
import glob
for p in glob.glob('./data/output/*.txt'):
    result = makeDataFrame(p)
    df = df.append(result, ignore_index=True)


In [ ]:
df

## データのクリーニング
冗長な空白を削除したり句読点をクリーンアップしたりして軽いデータ クリーニングを行い、トークン化の準備をします。

In [ ]:
pd.options.mode.chained_assignment = None #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#evaluation-order-matters

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df['content']= df["content"].apply(lambda x : normalize_text(x))

In [ ]:
df.iloc[0]["content"]

## ライブラリのインポートとモデルの一覧表示
[こちら](https://learn.microsoft.com/azure/cognitive-services/openai/tutorials/embeddings?tabs=command-line#retrieve-key-and-endpoint)を参照して Azure OpenAI Service のキーとエンドポイントの情報を取得します。

In [ ]:
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity
import requests

API_KEY = "<YOUR_AZURE_OPENAI_KEY>"
RESOURCE_ENDPOINT = "https://<YOUR_AZURE_OPENAI_SERVICE>.openai.azure.com/"

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2023-05-15"

url = openai.api_base + "/openai/deployments?api-version=2022-12-01"
r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)


このコマンドの出力は、デプロイしたモデルの数と種類によって異なります。 この場合は、**text-embedding-ada-002** のエントリがあることを確認する必要があります。 このモデルがない場合は、先に進む前にモデルをリソースにデプロイする必要があります。

## Embeddings を取得

In [ ]:
TEXT_SEARCH_DOC_MODEL   = 'text-embedding-ada-002'
ITEM_KEYWORD_EMBEDDING_FIELD = 'contentVector'

### 全件の取得には数分かかります

In [ ]:
%%time
df[ITEM_KEYWORD_EMBEDDING_FIELD] = df["content"].apply(lambda x : get_embedding(x, engine = TEXT_SEARCH_DOC_MODEL))

In [ ]:
df

## 参考 Parquet ファイルで保存&ロード
データの型の情報もファイルに保存したい人向け

In [ ]:
#df.to_parquet("./busho_dataset_embedding_ada.parquet")

In [ ]:
#import pyarrow.parquet as pq

#parqdf = pq.read_table("./busho_dataset_embedding_ada.parquet").to_pandas(integer_object_nulls=True)
#parqdf.head()

## DataFrame を JSON ファイルで保存

In [ ]:
df.to_json(f"busho_dataset_embedding_ada.json", orient='records', force_ascii=False)

# 3. Azure Cognitive Search に登録

[こちら](https://learn.microsoft.com/azure/search/search-get-started-rest#copy-a-key-and-url)を参照して Azure Cognitive Search に接続するためのキーを取得します。

In [ ]:
import requests
import json as JSON

# Azure Cognitive Search のインデックス名と API キー
search_service_name = "<YOUR_AZURE_COGNITIVE_SEARCH_SERVICE>"
index_name = "<YOUR_AZURE_COGNITIVE_SEARCH_INDEX>"
api_version = "2023-07-01-Preview"
api_key = "<YOUR_AZURE_COGNITIVE_SEARCH_APIKEY>"

# インデックス作成の REST API の URL
createindex_url = "https://{0}.search.windows.net/indexes/{1}?api-version={2}".format(search_service_name, index_name, api_version)

# データをアップロードするための REST API の URL
base_url = "https://{0}.search.windows.net/indexes/{1}/docs/index?api-version={2}".format(search_service_name, index_name, api_version)

## ベクトルインデックスの作成

In [ ]:
url = createindex_url

with open("./create_vector_index_with_semantic.json", 'r', encoding="utf-8") as f:
    jsondata = JSON.load(f)
    response = requests.put(url, headers={"Content-Type": "application/json", "api-key": api_key}, json=jsondata)
    print(response.status_code, response.text)

## ベクトルインデックスに JSON の内容を登録

In [ ]:
files = glob.glob('./busho_dataset_embedding_ada.json')

for file in files:
    print(file)
    with open(file, 'r', encoding="utf-8") as f:
        data = JSON.load(f)
    
        url = base_url 
        jsondata = {
            "value": data
        }
        response = requests.post(url, headers={"Content-Type": "application/json", "api-key": api_key}, json=jsondata)
        print(response.status_code)
        #print(response.text)
        

In [ ]:
print(response.text)